# Reporte diario promesas realizadas durante el dia

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import date
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta = '/home/estadistico/Documents/Erick/Reportes diarios/Promesas Diarias'

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Asignamos valores a los parametros \n",
today = date.today().strftime('%Y%m%d')
mes = 'Noviembre2020'

In [5]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

# Liverpool

In [6]:
#Hacemos la consulta referente a las gestiones de Liverpool
sql_gest_liv = cursor.callproc('liverpool_rpt_gestiones_detallado', [today,today])
for result in cursor.stored_results():
    gestion_liv = pd.DataFrame(result.fetchall())
gestion_liv.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv = gestion_liv.loc[gestion_liv['dictamen']=='PROMESA']
gestion_liv['credito'] = pd.to_numeric(gestion_liv['credito'])
gestion_liv.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,fecha_promesa,monto_promesa,comentarios
38,9245819,12,2,1400000003201237,ALEJANDRA MORA MIRANDA,2212384546,celular,2020-12-02 11:15:08,GARCIAFLOMAD,GARCIA FLORES OMAR ADRIAN,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-02,1986.98,"2212384546// TT REALIZA PAGO DE $1,986.98 CON..."
62,9245391,12,1,4178490011284255,YADIRA GUADALUPE OCHOA ALVAREZ,6622292466,celular,2020-12-02 10:55:08,GUZJABE,BERENICE GUZMAN JAIME,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,2000.00,(6622292466) 6622292466- promesa de pago parci...
87,9244669,12,2,1400000004200345,MARIA GUADALUPE GONZALEZ MORENO,5549455963,celular,2020-12-02 10:12:58,GARCIAFLOMAD,GARCIA FLORES OMAR ADRIAN,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-02,2716.00,5549455963// TT HACE UN PAGO LIQUIDACIÃN POR ...


In [7]:
#Hacemos la consulta de la asignacion
sql_asig_liv = cursor.callproc('liverpool_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_liv = pd.DataFrame(result.fetchall())
asignacion_liv = asignacion_liv.iloc[:,[0,1,3,24,6,26]]
asignacion_liv.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_liv['credito'] = pd.to_numeric(asignacion_liv['credito'])
asignacion_liv.head(3)

,firma_id,unegocio_id,credito,estado,rfc,division
0,12,0,10000001017274,MEX,MAMA720814JK8,DEPN
1,12,0,10000005443864,TAMPS,HEGD7610029Z4,DEPN
2,12,0,10000005657620,PUE,LADF4609143I1,SER4


In [8]:
#Hacemos el consolidado 
consolidado_liv = pd.merge(gestion_liv,asignacion_liv,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,fecha_promesa,monto_promesa,comentarios,estado,rfc,division
0,9245819,12,2,1400000003201237,ALEJANDRA MORA MIRANDA,2212384546,celular,2020-12-02 11:15:08,GARCIAFLOMAD,GARCIA FLORES OMAR ADRIAN,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-02,1986.98,"2212384546// TT REALIZA PAGO DE $1,986.98 CON...",PUE,MOMA9705102M5,SB02
1,9245391,12,1,4178490011284255,YADIRA GUADALUPE OCHOA ALVAREZ,6622292466,celular,2020-12-02 10:55:08,GUZJABE,BERENICE GUZMAN JAIME,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,2000.00,(6622292466) 6622292466- promesa de pago parci...,SON,OOAY670208R40,REM2
2,9244669,12,2,1400000004200345,MARIA GUADALUPE GONZALEZ MORENO,5549455963,celular,2020-12-02 10:12:58,GARCIAFLOMAD,GARCIA FLORES OMAR ADRIAN,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-02,2716.00,5549455963// TT HACE UN PAGO LIQUIDACIÃN POR ...,MEX,GOMG920522HL7,SB02


In [9]:
#Obtenemos datos de las promesas de Liverpool
pivot_liv = pd.pivot_table(consolidado_liv,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_liv = pd.DataFrame(pivot_liv.to_records())
pivot_liv = pivot_liv.iloc[:,[0,1,4]]
pivot_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_liv.fillna(0,inplace=True)
#pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_liv

,Division,NumeroPromesas,SumaPromesas
0,REM2,2,3754.00
1,REM4,3,6434.00
2,SB02,5,15824.98
3,pren,2,11833.00
4,All,12,37845.98


In [10]:
#Vemos las promesas reales eliminando duplicados
consolidado_liv2 = consolidado_liv.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_liv2 = pd.pivot_table(consolidado_liv2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_liv2 = pd.DataFrame(pivot_liv2.to_records())
pivot_liv2 = pivot_liv2.iloc[:,[0,1,4]]
pivot_liv2.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_liv2.fillna(0,inplace=True)
#pivot_liv2[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_liv2[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_liv2

,Dictamen,NumeroPromesas,SumaPromesas
0,REM2,2,3754.00
1,REM4,3,6434.00
2,SB02,5,15824.98
3,pren,2,11833.00
4,All,12,37845.98


# Bradesco

In [11]:
#Hacemos la consulta referente a las gestiones de Bradesco
sql_gest_brad = cursor.callproc('bradescard_rpt_gestiones_detallado', [today,today])
for result in cursor.stored_results():
    gestion_brad = pd.DataFrame(result.fetchall())
gestion_brad.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad = gestion_brad.loc[gestion_brad['dictamen']=='PROMESA']
gestion_brad['credito'] = pd.to_numeric(gestion_brad['credito'])
gestion_brad.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,accion_resultado,fecha_promesa,monto_promesa,comentarios
72,9246146,1,3,5063291381201019,DE LA ROSA LEYVA ARACELIS DEL CARMEN,9931913695,celular,2020-12-02 11:31:32,TRFIERROSFJ,TRON FIERROS FELIPE DE JESUS,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-12-02,1781.00,9931913695 TT INDICA LE LLEGO MENSAJE PARA LIQ...
130,9246201,1,3,4037500020930010,CARRASCO BELTRAN ADRIANA,5532777794,celular,2020-12-02 11:26:17,HDEZMELE,HERNANDEZ MEDEL LUIS ENRIQUE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,LLAMADA A TITULAR-PROMESA DE PAGO PARCIAL,2020-12-04,2000.00,5532777794 MENCIONA QUE NO TIENE TRABAJO ESTA...
232,9245923,1,3,5063291339468017,HARO JIMENEZ RICARDO,3313725048,nextel,2020-12-02 11:18:52,DOMINGUEZHEANIT,DOMINGUEZ HERNANDEZ ANAID ITZEL,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-12-04,1573.00,3313725048 IN/ TT CONFIRMA PP POR $1573.00 EN...


In [12]:
#Convertimos a numero las columnas que necesitemos
gestion_brad[cols] = gestion_brad[cols].apply(pd.to_numeric, errors='coerce')

In [13]:
#Hacemos la consulta de la asignacion
sql_asig_brad = cursor.callproc('bradescard_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_brad = pd.DataFrame(result.fetchall())
asignacion_brad = asignacion_brad.iloc[:,[0,1,3,24,6,26]]
asignacion_brad.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_brad.head(3)

,firma_id,unegocio_id,credito,estado,rfc,division
0,1,3,2869000138080177,YUCATAN,JURM8405047J801,C0 5PV
1,1,3,2869000240445953,VERACRUZ,TOHI650826EZ601,C0 4PV
2,1,3,2869000320452929,PUEBLA,HERL821006J1401,C0 4PV


In [14]:
#Hacemos la union de asignacion y promesas
consolidado_brad = pd.merge(gestion_brad,asignacion_brad,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,accion_resultado,fecha_promesa,monto_promesa,comentarios,estado,rfc,division
0,9246146,1,3,5063291381201019,DE LA ROSA LEYVA ARACELIS DEL CARMEN,9931913695,celular,2020-12-02 11:31:32,TRFIERROSFJ,TRON FIERROS FELIPE DE JESUS,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-12-02,1781.0,9931913695 TT INDICA LE LLEGO MENSAJE PARA LIQ...,TABASCO,ROLA960101SZ501,C0 4PV
1,9246201,1,3,4037500020930010,CARRASCO BELTRAN ADRIANA,5532777794,celular,2020-12-02 11:26:17,HDEZMELE,HERNANDEZ MEDEL LUIS ENRIQUE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,LLAMADA A TITULAR-PROMESA DE PAGO PARCIAL,2020-12-04,2000.0,5532777794 MENCIONA QUE NO TIENE TRABAJO ESTA...,CIUDAD DE MÃXICO,CABA851025BQ401,C0 6PV
2,9245923,1,3,5063291339468017,HARO JIMENEZ RICARDO,3313725048,nextel,2020-12-02 11:18:52,DOMINGUEZHEANIT,DOMINGUEZ HERNANDEZ ANAID ITZEL,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-12-04,1573.0,3313725048 IN/ TT CONFIRMA PP POR $1573.00 EN...,JALISCO,HAJR000611LM801,C0 4PV


In [15]:
#Obtenemos datos de las promesas de Bradesco
pivot_brad = pd.pivot_table(consolidado_brad,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_brad = pd.DataFrame(pivot_brad.to_records())
pivot_brad = pivot_brad.iloc[:,[0,1,4]]
pivot_brad.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_brad.fillna(0,inplace=True)
#pivot_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad

,Division,NumeroPromesas,SumaPromesas
0,C0 4PV,13,19785.0
1,C0 5PV,1,1457.0
2,C0 6PV,2,2700.0
3,C1,13,20476.0
4,C2,1,1000.0
5,C3,3,3304.0
6,All,33,48722.0


In [16]:
#Vemos las promesas reales eliminando duplicados
consolidado_brad2 = consolidado_brad.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_brad2 = pd.pivot_table(consolidado_brad2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_brad2 = pd.DataFrame(pivot_brad2.to_records())
pivot_brad2 = pivot_brad2.iloc[:,[0,1,4]]
pivot_brad2.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_brad2.fillna(0,inplace=True)
#pivot_brad2[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad2[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad2

,Division,NumeroPromesas,SumaPromesas
0,C0 4PV,13,19785.0
1,C0 5PV,1,1457.0
2,C0 6PV,2,2700.0
3,C1,13,20476.0
4,C2,1,1000.0
5,C3,3,3304.0
6,All,33,48722.0


# Credifiel

In [17]:
#Hacemos la consulta referente a las gestiones de Credifiel
sql_gest_cred = cursor.callproc('credifiel_rpt_gestiones_detallado', [today,today])
for result in cursor.stored_results():
    gestion_cred = pd.DataFrame(result.fetchall())
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
cols = ['monto_promesa','unegocio_id']
gestion_cred[cols] = gestion_cred[cols].apply(pd.to_numeric, errors='coerce')
gestion_cred = gestion_cred.loc[gestion_cred['monto_promesa']>0]
gestion_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen
25,9241276,1,2020-12-02,07:34:59,CR-418505,TZOMPANTZI TEXIS JUANA,5548632739,GONMORALESBM,GONZALEZ MORALES BRENDA MARLEN,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,1090.0,5548632739 TT REALIZARA PAGO 4 POR $1090 EL 15...,PDP,Promesa de Pago,CONTACTO
99,9242467,1,2020-12-02,07:45:17,CR-532876,MARQUEZ ARREDONDO ROBERTO DE JESUS,7831177323,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,ENTRADA WHATS APP,PROMESA DE PAGO PARCIAL,2020-12-15,4196.3,7831177323 TT REALIZARA PAGO EL DIA 15/12/2020...,PDP,Promesa de Pago,CONTACTO


In [18]:
#Hacemos la consulta de la asignacion
sql_asig_cred = cursor.callproc('credifiel_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_cred = pd.DataFrame(result.fetchall())
asignacion_cred = asignacion_cred.iloc[:,[1,3,32]]
asignacion_cred.columns = ['unegocio_id','credito','division']
#asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_cred.head(3)

,unegocio_id,credito,division
0,1,CR-101922,COBRANZA EXTRAJUDICIAL
1,1,CR-101949,LIQUIDAR
2,1,CR-101952,LIQUIDAR


In [19]:
#Hacemos la union de asignacion y promesas
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',on=['unegocio_id','credito'])
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen,division
0,9241276,1,2020-12-02,07:34:59,CR-418505,TZOMPANTZI TEXIS JUANA,5548632739,GONMORALESBM,GONZALEZ MORALES BRENDA MARLEN,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,1090.0,5548632739 TT REALIZARA PAGO 4 POR $1090 EL 15...,PDP,Promesa de Pago,CONTACTO,
1,9242467,1,2020-12-02,07:45:17,CR-532876,MARQUEZ ARREDONDO ROBERTO DE JESUS,7831177323,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,ENTRADA WHATS APP,PROMESA DE PAGO PARCIAL,2020-12-15,4196.3,7831177323 TT REALIZARA PAGO EL DIA 15/12/2020...,PDP,Promesa de Pago,CONTACTO,COBRANZA EXTRAJUDICIAL


In [20]:
#Obtenemos datos de las promesas de Credifiel
pivot_cred = pd.pivot_table(consolidado_cred,index=['division'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_cred = pd.DataFrame(pivot_cred.to_records())
pivot_cred = pivot_cred.iloc[:,[0,3,2]]
pivot_cred.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_cred.fillna(0,inplace=True)
pivot_cred = pivot_cred.loc[pivot_cred['SumaPromesas']>0]
#pivot_gestion_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred

,Division,NumeroPromesas,SumaPromesas
0,,1,1090.0
1,COBRANZA EXTRAJUDICIAL,1,4196.3


In [21]:
#Vemos las promesas reales eliminando duplicados
consolidado_cred2 = consolidado_cred.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_cred2 = pd.pivot_table(consolidado_cred2,index=['division'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_cred2 = pd.DataFrame(pivot_cred2.to_records())
pivot_cred2 = pivot_cred2.iloc[:,[0,3,2]]
pivot_cred2.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_cred2.fillna(0,inplace=True)
pivot_cred2 = pivot_cred2.loc[pivot_cred2['SumaPromesas']>0]
#pivot_gestion_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred2

,Division,NumeroPromesas,SumaPromesas
0,,1,1090.0
1,COBRANZA EXTRAJUDICIAL,1,4196.3


# Banco Azteca

In [22]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
sql_gest_baz = cursor.callproc('baz_rpt_gestiones_detallado', [today,today])
for result in cursor.stored_results():
     gestion_baz = pd.DataFrame(result.fetchall())
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
gestion_baz.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion_resultado,fecha_promesa,monto_promesa,comentarios
43,9246095,7,1,1-20-2386-5686,JORGE LUIS URREA FLORES,5561118790,celular,2020-12-02 07:58:33,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-07,491.00,5561118790 TT REALIZARA PAGO SEMANAL POR 491 X...
44,9246094,7,1,1-1-2684-114992,ALEXIS ABIRAM MORALES VERDUGO,9626968554,celular,2020-12-02 07:59:30,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-07,480.00,9626968554 pagos sem de 480
46,9246092,7,1,1-1-6606-18281,ISRAEL RAMIRO AQUINO MONROY,5571868040,oficina,2020-12-02 08:00:08,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,PROMESA,GESTION TELEFONICA -PROMESA DE PAGO PARCIAL,2020-12-07,431.00,5571868040 5526191333 5537539557 pagos sem de 431


In [23]:
sample_cols_to_keep =['#FIIDCAMPANA', 'FIPAIS', 'FICANAL','FISUCURSAL','FIFOLIO','FISEMATRASMAX','FNSALDO','FNSALDOCAPITAL','FNPAGOREQ','FCTEL1','FCTIPO1','FNDIA_PAGO','FIDIASATRASOMAX','FNCAPPAGODISP','FNABONOSEMANAL','FNCAPACIDADPAGO','FILCRACTIVA','FDFECPROXPAG','FNTASAINT','CP','fcnombreos','fcappaternoos','FITERRITORIO','FCDESCTERRITORIO','FIZONA','FCDESCZONA','FIREGION','FCDESCREGION','FIGERENCIA','FCDESCGERENCIA','FCBESTTIMETOCALL','FECHA']
df_iter = pd.read_csv('/home/estadistico/Documents/Erick/Banco Azteca/Asignacion_csv/Asignacion banco azteca '+mes+'.csv', chunksize=20000, usecols=sample_cols_to_keep) 
df_lst = [] 
for df_ in df_iter: 
            tmp_df = (df_.rename(columns={col: col.lower() for col in df_.columns}))
            df_lst += [tmp_df.copy()] 
asignacion_baz = pd.concat(df_lst)

In [24]:
asignacion_baz['credito'] = asignacion_baz['fipais'].map(str)+'-'+asignacion_baz['ficanal'].map(str)+'-'+asignacion_baz['fisucursal'].map(str)+'-'+asignacion_baz['fifolio'].map(str)
asignacion_baz.head(3)

,#fiidcampana,fipais,ficanal,fisucursal,fifolio,fisematrasmax,fnsaldo,fnsaldocapital,fnpagoreq,fctel1,...,fcdescterritorio,fizona,fcdesczona,firegion,fcdescregion,figerencia,fcdescgerencia,fcbesttimetocall,fecha,credito
0,160,1,1,657,83392,1,28128,4199,249,6371181906,...,9828 TERRITORIO NORTE,9875,9875 HERMOSILLO,8262,8262 RCR HERMOSILLO 3,8305,8305 GCC CABORCA,NaN,11/01/2020,1-1-657-83392
1,160,1,51,8032,104288,1,31062,4500,414,6565811031,...,9828 TERRITORIO NORTE,9871,9871 CHIHUAHUA,9324,9324 RCR CHIHUAHUA 4,9618,9618 GCC OASIS REVOLUCION,NaN,11/01/2020,1-51-8032-104288
2,160,1,1,2020,1379,1,63550,492,649,6531392132,...,9828 TERRITORIO NORTE,9878,9878 TIJUANA,8261,8261 RCR TIJUANA 3,8304,8304 GCC SAN LUIS RIO COLORADO,NaN,11/01/2020,1-1-2020-1379


In [25]:
consolidado_baz = pd.merge(gestion_baz,asignacion_baz,how='left',on='credito')
cols = ['monto_promesa','#fiidcampana']
consolidado_baz[cols] = consolidado_baz[cols].apply(pd.to_numeric, errors='coerce')
consolidado_baz.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,...,fiterritorio,fcdescterritorio,fizona,fcdesczona,firegion,fcdescregion,figerencia,fcdescgerencia,fcbesttimetocall,fecha
0,9246095,7,1,1-20-2386-5686,JORGE LUIS URREA FLORES,5561118790,celular,2020-12-02 07:58:33,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9246094,7,1,1-1-2684-114992,ALEXIS ABIRAM MORALES VERDUGO,9626968554,celular,2020-12-02 07:59:30,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9246092,7,1,1-1-6606-18281,ISRAEL RAMIRO AQUINO MONROY,5571868040,oficina,2020-12-02 08:00:08,HEBALTAZARAL,HERNANDEZ BALTAZAR ALFREDO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#Obtenemos datos de las promesas de Banco Azteca
pivot_baz = pd.pivot_table(consolidado_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_baz = pd.DataFrame(pivot_baz.to_records())
pivot_baz = pivot_baz.iloc[:,[0,3,2]]
pivot_baz.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_baz.fillna(0,inplace=True)
#pivot_baz[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_baz[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_baz

,Dictamen,NumeroPromesas,SumaPromesas
0,PROMESA,37,25846.0


In [27]:
#Vemos las promesas reales eliminando duplicados
consolidado_baz2 = consolidado_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_baz2 = pd.pivot_table(consolidado_baz2,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_baz2 = pd.DataFrame(pivot_baz2.to_records())
pivot_baz2 = pivot_baz2.iloc[:,[0,3,2]]
pivot_baz2.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_baz2.fillna(0,inplace=True)
#pivot_baz2[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_baz2[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_baz2

,Dictamen,NumeroPromesas,SumaPromesas
0,PROMESA,37,25846.0


In [28]:
writer = pd.ExcelWriter(''+ruta+'/Reporte Promesas Diarias '+today+'.xlsx', engine='xlsxwriter')
pivot_liv.to_excel(writer,'PromDiariaLivBruto',index=False,header=True)
pivot_liv2.to_excel(writer,'PromDiariaLivReal',index=False,header=True)
pivot_brad.to_excel(writer,'PromDiariaBradBruto',index=False,header=True)
pivot_brad2.to_excel(writer,'PromDiariaBradReal',index=False,header=True)
pivot_cred.to_excel(writer,'PromDiariaCredBruto',index=False,header=True)
pivot_cred2.to_excel(writer,'PromDiariaCredReal',index=False,header=True)
pivot_baz.to_excel(writer,'PromDiariaBAZBruto',index=False,header=True)
pivot_baz2.to_excel(writer,'PromDiariaBAZReal',index=False,header=True)

writer.save()
writer.close()